# Train and deploy a model

> [!NOTE] Must use Python 3.10 SDK V2 for this demo.

## Train a model

Initiate a connection to the Azure ML workspace and set up MLflow for tracking.

In [1]:
## Train a model

# Handle to the workspace
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
import mlflow

ml_client = MLClient.from_config(
    DefaultAzureCredential()
)

# Gather MLflow URI information from workspace
azureml_mlflow_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_mlflow_uri)

Found the config file in: /config.json


Import necessary libraries and set up the experiment in MLflow.

In [2]:
# Import python packages
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
import numpy as np
import os

experiment_name = "Monitoring-Models-Experiment"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='', creation_time=1695979876979, experiment_id='25664d2b-30d0-447f-a683-d167b73e4c85', last_update_time=None, lifecycle_stage='active', name='Monitoring-Models-Experiment', tags={}>

Load the dataset, convert it to a Pandas DataFrame, and prepare the directory for model saving.

In [3]:
import mltable

# iterate over all versions of the data asset

data_asset = ml_client.data.get("diabetes-mltable-dev", label="latest")

tbl = mltable.load(data_asset.path)

df = tbl.to_pandas_dataframe()
df

model_path = "./models/monitoring"
os.makedirs(model_path, exist_ok=True)

Start logging the training process in MLflow, train a Decision Tree model, and log the model performance metrics.

In [4]:
# delete model directory if it exists
import shutil
if os.path.exists(model_path):
    shutil.rmtree(model_path)

# Start Logging
mlflow.start_run()

# Enable autologging (optional)
# mlflow.sklearn.autolog()

diabetes = df

# Breaking up data into input/target features
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Breaking data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Training a model:
model = DecisionTreeClassifier().fit(X_train, y_train)

# Calculating performance and logging them
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
mlflow.log_metric('Accuracy', float(acc))

y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
mlflow.log_metric('AUC', float(auc))

Infer the model signature, register the model to the workspace, and save the model to a file.

In [5]:
from mlflow.models import infer_signature

signature = infer_signature(X_test, y_hat)

# Registering the model to the workspace
print("Registering the model via MLFlow")
mlflow.sklearn.log_model(
    sk_model=model,
    registered_model_name="monitoring-diabetes-clr-mdl",
    artifact_path="model",
    signature=signature,
)

# Saving the model to a file
mlflow.sklearn.save_model(
    sk_model=model, 
    path=model_path,
    signature=signature
)

# Stop logging
mlflow.end_run()


Registering the model via MLFlow


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'monitoring-diabetes-clr-mdl' already exists. Creating a new version of this model...
2023/10/13 17:42:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: monitoring-diabetes-clr-mdl, version 14
Created version '14' of model 'monitoring-diabetes-clr-mdl'.


## Deploy a Model

After training and registering the model, it's time to deploy it. You can verify the latest version of your registered model on the Models page on Azure ML studio, or use the code below to retrieve the latest version number.

<img title="ws-model-registered" src="Media/ws-model-registered.png">

In [6]:
## Deploy a Model

# Name the model you registered earlier in the training script
registered_model_name = "monitoring-diabetes-clr-mdl"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)


Import the necessary entities for creating an online endpoint and defining an online deployment.

In [7]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
)

Create a unique online endpoint name, and define the online endpoint with necessary configurations.

In [8]:
import datetime

# Creating a unique online endpoint name with current datetime to avoid conflicts
online_endpoint_name = "monitor-diabetes" + datetime.datetime.now().strftime("%m%d%H%M%f")

# Create an online endpoint
endpoint = ManagedOnlineEndpoint(
  name=online_endpoint_name,
  description="This is a diabetes classifier online endpoint",
  auth_mode="key",
  tags={
      "training_dataset": "diabetes-data",
      "model_type": "sklearn.DecisionTreeClassifier",
      "purpose": "demonstration"
  },
)

endpoint = ml_client.begin_create_or_update(endpoint)

In [11]:
endpoint.status()

'Succeeded'

> [!NOTE] Make sure to wait for a notification that the endpoint has completed deployment.

<img title="Endpoint Deployment" src="Media/endpoint-deployment-succeeded.png">

<img title="Endpoint Deployment" src="Media/endpoint-deployment-succeeded_notification.png">

### Deploy the model to the endpoint

Upon endpoint creation, proceed to deploy the model. You can manage multiple deployments per endpoint, directing traffic via specified rules. In this scenario, create a single deployment to handle all incoming traffic. The color-naming convention (e.g., blue, green, red) for deployments is arbitrary but helps in distinguishing them.

> [!NOTE]
> Replace the endpoint name with the endpoint configured in the previous step.
> Expect this deployment to take approximately 6 to 8 minutes.

In [12]:
import time

model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

notebook_deployment = ManagedOnlineDeployment(
    name="notebook-deployment",
    #Replace the endpoint name below
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
    app_insights_enabled=True,
)

try:
    ml_client.online_deployments.begin_create_or_update(notebook_deployment)
    print("Creating notebook deployment on endpoint")
except Exception as e:
    print(e)
    print("Waiting 3 Minutes...")
    time.sleep(180)
    ml_client.online_deployments.begin_create_or_update(notebook_deployment)
    print("Creating notebook deployment on endpoint")


Check: endpoint monitor-diabetes10131742234249 exists


Creating notebook deployment on endpoint
...

> [!Note] You can view the current provision status on the red deployment on the **Endpoint** Tab on the right hand side. 

Remember to wait for the deployment to be completed before moving to the next cell.

<img title="Endpoint Provision Pending Status" src="Media/endpoint-provision-status-pending.png">


### Managing Endpoints and Deployments
Retrieve details about the online endpoint and existing traffic rules. Update the traffic rules as needed and apply the changes to the endpoint.

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# Existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

# Update traffic rules
endpoint.traffic = {"red": 100}

# Apply the updated traffic rules to the endpoint
ml_client.begin_create_or_update(endpoint)

Upon completion, the endpoint's provision status should reflect the changes.

<img title="Endpoint Provision Status" src="Media/endpoint-provision-status.png">

## Use the Model

Make sure to capture the API key from the **Endpoint** '**Consume**' page. Replace it with the code below for `api_key`


<img title="Endpoint Details" src="Media/endpoint-details-page.png">



In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

#generating new data to use in querying
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
        [4,96,83,26,34,52.94533137,0.160199188,53],
        [1,125,83,41,235,19.65795152,0.150529189,23],
        [3,106,83,39,223,31.77645097,0.877332438,22],
        [0,148,58,11,179,39.19207553,0.160829008,45]]

data =  {"input_data": x_new}

body = str.encode(json.dumps(data))

url = endpoint.scoring_uri
api_key = 'your-api-key'  # Replace this with the API key for the web service
headers = {
    'Content-Type':'application/json', 
    'Authorization':('Bearer '+ api_key), 
    'azureml-model-deployment': 'red'  # Directs request to the 'red' deployment
}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    predicted_classes = json.loads(result.decode())
    for i in range(len(x_new)):
        print(f"Patient {x_new[i]}: {predicted_classes[i]}")
except urllib.error.HTTPError as error:
    print(f"The request failed with status code: {error.code}")
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

## Conclusion

Navigate back to the Azure Portal to observe the results in Application Insights.

<img src="Media/endpoint-deployment-metrics.png">

Alternatively, inspect the Application Insight instance created alongside the Azure Machine Learning Workspace.

<img src="Media/azure-app-insight.png">

To delve into transactions, navigate to the **Investigate** category.

<img src="Media/azure-app-insight-transaction.png">

For more detailed analysis, select a specific result.

<img src="Media/azure-app-insight-transaction-results.png">

Under the **Performance** tab, select the desired operation to view logs.

<img src="Media/azure-app-insight-performance.png">

Explore the query results from Azure Log Analytics.

<img src="Media/logs-kql-commands.png">

The steps above facilitate the usage of your model for predictions and monitoring through Azure's robust logging and analytics tools.